In [1]:
import tensorflow
import numpy
import matplotlib

print(tensorflow.__version__)
print(numpy.__version__)
print(matplotlib.__version__)

2025-01-20 20:18:17.463189: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-20 20:18:17.481570: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-20 20:18:17.481588: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-20 20:18:17.481601: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-20 20:18:17.485207: I tensorflow/core/platform/cpu_feature_g

2.14.0
1.26.0
3.9.3


In [2]:
import os

data_dir = os.getcwd()
kor_path = data_dir+"/korean-english-park.train.ko"
eng_path = data_dir+"/korean-english-park.train.en"

# 데이터 정제 및 토큰화
def clean_corpus(kor_path, eng_path):
    with open(kor_path, "r") as f:
        kor = f.read().splitlines()
    with open(eng_path, "r") as f:
        eng = f.read().splitlines()
    
    assert len(kor) == len(eng)

    clean_set = set()
    for k, e in zip(kor, eng):
        clean_set.add(k + '\t' + e)
    cleaned_corpus = sorted(list(clean_set))
    
    return cleaned_corpus

cleaned_corpus = clean_corpus(kor_path, eng_path)

print(cleaned_corpus[0])

"" 이번 김 위원장의 방중은 딕 체니 미국 부통령이 베이징을 방문, 중국에 북한의 핵 위협 완화를 요구한 직후 이루어졌다.	Kim's visit comes just after Vice President Dick Cheney traveled to Beijing and urged China to do more to defuse Pyongyang's nuclear threat.


In [3]:
import re

def preprocess_sentence(sentence):
    # 1. 소문자로 변환
    sentence = sentence.lower()
    # 2. 알파벳, 문장부호, 한글만 남기고 모두 제거
    sentence = re.sub(r"[^a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣.,!?']", " ", sentence)
    # 3. 문장부호 양옆에 공백 추가
    sentence = re.sub(r"([.,!?])", r" \1 ", sentence)
    # 4. 여러 공백을 하나의 공백으로 변환
    sentence = re.sub(r"\s+", " ", sentence)
    # 5. 문장 앞뒤의 공백 제거
    sentence = sentence.strip()

    return sentence

for sentence in cleaned_corpus[0].split('\t'):
    print(preprocess_sentence(sentence))

이번 김 위원장의 방중은 딕 체니 미국 부통령이 베이징을 방문 , 중국에 북한의 핵 위협 완화를 요구한 직후 이루어졌다 .
kim's visit comes just after vice president dick cheney traveled to beijing and urged china to do more to defuse pyongyang's nuclear threat .


In [4]:
import sentencepiece as spm

# Sentencepiece를 활용하여 학습한 tokenizer를 생성합니다
def generate_tokenizer(corpus,
                       vocab_size=20000,
                       lang="ko",
                       pad_id=0,
                       bos_id=1,
                       eos_id=2,
                       unk_id=3):
    
    if lang == 'ko':
        spm.SentencePieceTrainer.Train(
            f'--input={kor_path} --model_prefix={lang}_spm --vocab_size={vocab_size}'
            f'--pad_id={pad_id} --bos_id={bos_id} --eos_id={eos_id} --unk_id={unk_id}'
        )
    elif lang == 'en':
        spm.SentencePieceTrainer.Train(
            f'--input={eng_path} --model_prefix={lang}_spm --vocab_size={vocab_size}'
            f'--pad_id={pad_id} --bos_id={bos_id} --eos_id={eos_id} --unk_id={unk_id}'
        )
    else: 
        print('lang 오류')
        
    # 학습된 모델 로드
    tokenizer = spm.SentencePieceProcessor()
    tokenizer.Load(f"{lang}_spm.model")

    return tokenizer


SRC_VOCAB_SIZE = TGT_VOCAB_SIZE = 20000

eng_corpus = []
kor_corpus = []

for pair in cleaned_corpus:
    k, e = pair.split("\t")

    kor_corpus.append(preprocess_sentence(k))
    eng_corpus.append(preprocess_sentence(e))

ko_tokenizer = generate_tokenizer(kor_corpus, SRC_VOCAB_SIZE, "ko")
en_tokenizer = generate_tokenizer(eng_corpus, TGT_VOCAB_SIZE, "en")
en_tokenizer.set_encode_extra_options("bos:eos")

sentencepiece_trainer.cc(178) LOG(INFO) Running command: --input=/workspace/aiffel/transformer/korean-english-park.train.ko --model_prefix=ko_spm --vocab_size=20000--pad_id=0 --bos_id=1 --eos_id=2 --unk_id=3
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: /workspace/aiffel/transformer/korean-english-park.train.ko
  input_format: 
  model_prefix: ko_spm
  model_type: UNIGRAM
  vocab_size: 20000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 

True

In [5]:
# Test 결과 확인
print("KO Tokenizer 예제:", ko_tokenizer.EncodeAsPieces("안녕하세요, 만나서 반갑습니다."))
print("EN Tokenizer 예제:", en_tokenizer.EncodeAsPieces("Hello, nice to meet you."))

KO Tokenizer 예제: ['▁안', '녕', '하세요', ',', '▁만나', '서', '▁반', '갑', '습니다', '.']
EN Tokenizer 예제: ['<s>', '▁', 'Hello', ',', '▁nice', '▁to', '▁meet', '▁you', '.', '</s>']


In [6]:
from tqdm.notebook import tqdm
import tensorflow as tf

src_corpus = []
tgt_corpus = []

# 한국어와 영어 코퍼스의 길이 확인
assert len(kor_corpus) == len(eng_corpus), "한국어와 영어 코퍼스의 길이가 일치하지 않습니다."

# 토큰 길이가 50 이하인 문장만 선별
for idx in tqdm(range(len(kor_corpus))):
    # 한국어와 영어 문장을 각각 토큰화
    src_tokens = ko_tokenizer.EncodeAsIds(kor_corpus[idx])
    tgt_tokens = en_tokenizer.EncodeAsIds(eng_corpus[idx])

    # 토큰 길이가 50 이하인 경우만 저장
    if len(src_tokens) <= 50 and len(tgt_tokens) <= 50:
        src_corpus.append(src_tokens)
        tgt_corpus.append(tgt_tokens)

# 패딩 처리를 완료하여 학습용 데이터를 생성
enc_train = tf.keras.preprocessing.sequence.pad_sequences(
    src_corpus, padding='post')
dec_train = tf.keras.preprocessing.sequence.pad_sequences(
    tgt_corpus, padding='post')

# 결과 출력
print(f"전체 데이터 {len(kor_corpus)} 중 {len(src_corpus)}개의 문장이 선택되었습니다.")
print(f"enc_train shape: {enc_train.shape}")
print(f"dec_train shape: {dec_train.shape}")

  0%|          | 0/78968 [00:00<?, ?it/s]

전체 데이터 78968 중 64679개의 문장이 선택되었습니다.
enc_train shape: (64679, 50)
dec_train shape: (64679, 50)


In [7]:
enc_train[0]

array([   53,   739,  3811,     6, 16370,    13,  4211,  2045,    49,
        8777,  1011,     4,   501,     5,    12,  4370,   103,     6,
         285,   856,  2453,     7,  3034,  2542,  4648,     5,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0], dtype=int32)

In [8]:
dec_train[0]

array([    1,    14,  3757,   104,    12,     6,   430,   866,   161,
          46,  1483,   181,  2852,  3119,    14,  2800,  5797,   703,
          23,     7,    32, 13051,    20,    11,    14,  2055, 11015,
         136,     7,   183,    60,     7, 11603,   484,  6310,  7808,
          12,     6,   140,  1122,    14,     4,     2,     0,     0,
           0,     0,     0,     0,     0], dtype=int32)

In [9]:
import numpy as np
import tensorflow as tf

def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, int(i) / d_model)

    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i)
                              for pos_i in range(pos)])
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])
    return sinusoid_table


class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        self.depth = d_model // self.num_heads

        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)

        self.linear = tf.keras.layers.Dense(d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)

        scaled_qk = QK / tf.math.sqrt(d_k)

        if mask is not None:
            scaled_qk += (mask * -1e9)

        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        out = tf.matmul(attentions, V)

        return out, attentions

    def split_heads(self, x):
        batch_size = x.shape[0]
        split_x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])

        return split_x

    def combine_heads(self, x):
        batch_size = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (batch_size, -1, self.d_model))

        return combined_x

    def call(self, Q, K, V, mask):
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)

        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)

        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask)

        out = self.combine_heads(out)
        out = self.linear(out)

        return out, attention_weights
    

class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.w_1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.w_2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        out = self.w_1(x)
        out = self.w_2(out)

        return out
    

class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout = tf.keras.layers.Dropout(dropout)

    def call(self, x, mask):
        """
        Multi-Head Attention
        """
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.dropout(out)
        out += residual

        """
        Position-Wise Feed Forward Network
        """
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.dropout(out)
        out += residual

        return out, enc_attn
    

class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout = tf.keras.layers.Dropout(dropout)

    def call(self, x, enc_out, causality_mask, padding_mask):
        """
        Masked Multi-Head Attention
        """
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.dropout(out)
        out += residual

        """
        Multi-Head Attention
        """
        residual = out
        out = self.norm_2(out)
        out, dec_enc_attn = self.enc_dec_attn(
            out, enc_out, enc_out, causality_mask)
        out = self.dropout(out)
        out += residual

        """
        Position-Wise Feed Forward Network
        """
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.dropout(out)
        out += residual

        return out, dec_attn, dec_enc_attn
    

class Encoder(tf.keras.Model):
    def __init__(self, n_layers, d_model, n_heads, d_ff, dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(
            d_model, n_heads, d_ff, dropout) for _ in range(n_layers)]

    def call(self, x, mask):
        out = x

        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)

        return out, enc_attns
    

class Decoder(tf.keras.Model):
    def __init__(self,
                 n_layers,
                 d_model,
                 n_heads,
                 d_ff,
                 dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout)
                           for _ in range(n_layers)]

    def call(self, x, enc_out, causality_mask, padding_mask):
        out = x

        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = \
                self.dec_layers[i](out, enc_out, causality_mask, padding_mask)

            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        return out, dec_attns, dec_enc_attns
    

class Transformer(tf.keras.Model):
    def __init__(self,
                 n_layers,
                 d_model,
                 n_heads,
                 d_ff,
                 src_vocab_size,
                 tgt_vocab_size,
                 pos_len,
                 dropout=0.2,
                 shared=True):
        super(Transformer, self).__init__()
        self.d_model = tf.cast(d_model, tf.float32)

        self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
        self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.dropout = tf.keras.layers.Dropout(dropout)

        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

        self.fc = tf.keras.layers.Dense(tgt_vocab_size)

        self.shared = shared

        if shared:
            self.fc.set_weights(tf.transpose(self.dec_emb.weights))

    def embedding(self, emb, x):
        seq_len = x.shape[1]
        out = emb(x)

        if self.shared:
            out *= tf.math.sqrt(self.d_model)

        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.dropout(out)

        return out

    def call(self, enc_in, dec_in, enc_mask, causality_mask, dec_mask):
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)

        enc_out, enc_attns = self.encoder(enc_in, enc_mask)

        dec_out, dec_attns, dec_enc_attns = \
            self.decoder(dec_in, enc_out, causality_mask, dec_mask)

        logits = self.fc(dec_out)

        return logits, enc_attns, dec_attns, dec_enc_attns

In [10]:
# Hyperparameters
NUM_LAYERS = 2            # Transformer layers
D_MODEL = 512             # Embedding dimension
NUM_HEADS = 8             # Number of attention heads
D_FF = 2048               # Feed forward network hidden layer size
SRC_VOCAB_SIZE = 20000     # Source vocabulary size (한국어)
TGT_VOCAB_SIZE = 20000     # Target vocabulary size (영어)
MAX_SEQ_LEN = 100         # Maximum sequence length
DROPOUT = 0.3             # Dropout rate

# Transformer 모델 선언
transformer = Transformer(
    n_layers=NUM_LAYERS,
    d_model=D_MODEL,
    n_heads=NUM_HEADS,
    d_ff=D_FF,
    src_vocab_size=SRC_VOCAB_SIZE,
    tgt_vocab_size=TGT_VOCAB_SIZE,
    pos_len=MAX_SEQ_LEN,
    dropout=DROPOUT,
    shared=False  # False로 설정해 독립적인 Embedding 사용
)

2025-01-20 20:18:28.155644: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-01-20 20:18:28.158075: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-01-20 20:18:28.158142: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [11]:
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()
        self.d_model = tf.cast(d_model, tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, tf.float32)  # step을 부동소수점으로 캐스팅
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


learning_rate = LearningRateScheduler(512)
optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9
)

In [12]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')


def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    # Masking 되지 않은 입력의 개수로 Scaling하는 과정
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [13]:
import tensorflow as tf


def generate_padding_mask(seq):
    """
    패딩 마스크를 생성합니다.
    seq에서 0(PAD)인 위치를 1로, 나머지를 0으로 설정합니다.
    """
    return tf.cast(tf.math.equal(seq, 0), tf.float32)[:, tf.newaxis, tf.newaxis, :]


def generate_look_ahead_mask(size):
    """
    룩어헤드 마스크를 생성합니다.
    디코더에서 현재와 이전의 토큰만 볼 수 있도록 상삼각 행렬을 생성합니다.
    """
    return 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)


def generate_masks(src, tgt):
    """
    Transformer 모델에서 필요한 마스크들을 생성합니다.
    Args:
        src (Tensor): 소스 입력 텐서
        tgt (Tensor): 타겟 입력 텐서
    Returns:
        enc_mask: 인코더 입력의 패딩 마스크
        dec_enc_mask: 디코더가 인코더 출력과 상호작용할 때 사용하는 패딩 마스크
        dec_mask: 디코더의 룩어헤드 + 패딩 마스크
    """
    # 인코더 패딩 마스크
    enc_mask = generate_padding_mask(src)

    # 디코더 패딩 마스크
    dec_enc_mask = generate_padding_mask(src)

    # 디코더 룩어헤드 + 패딩 마스크
    seq_len = tf.shape(tgt)[1]
    look_ahead_mask = generate_look_ahead_mask(seq_len)
    dec_pad_mask = generate_padding_mask(tgt)
    dec_mask = tf.maximum(look_ahead_mask, dec_pad_mask)

    return enc_mask, dec_enc_mask, dec_mask

In [14]:
@tf.function()
def train_step(src, tgt, model, optimizer):
    # 타겟 데이터에서 <BOS> 제거 (gold는 정답)
    gold = tgt[:, 1:]

    # 입력 마스크 생성
    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt)

    # GradientTape으로 손실 계산
    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = model(
            src, tgt, enc_mask, dec_enc_mask, dec_mask
        )
        loss = loss_function(gold, predictions[:, :-1])

    # 그래디언트 계산 및 가중치 업데이트
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, enc_attns, dec_attns, dec_enc_attns

In [15]:
import matplotlib.pyplot as plt

# Attention 시각화 함수
def visualize_attention(src, tgt, enc_attns, dec_attns, dec_enc_attns):
    def draw(data, ax, x="auto", y="auto"):
        import seaborn
        seaborn.heatmap(data,
                        square=True,
                        vmin=0.0, vmax=1.0,
                        cbar=False, ax=ax,
                        xticklabels=x,
                        yticklabels=y)

    for layer in range(0, 2, 1):
        fig, axs = plt.subplots(1, 4, figsize=(20, 10))
        print("Encoder Layer", layer + 1)
        for h in range(4):
            draw(enc_attns[layer][0, h, :len(src),
                 :len(src)], axs[h], src, src)
        plt.show()

    for layer in range(0, 2, 1):
        fig, axs = plt.subplots(1, 4, figsize=(20, 10))
        print("Decoder Self Layer", layer+1)
        for h in range(4):
            draw(dec_attns[layer][0, h, :len(tgt),
                 :len(tgt)], axs[h], tgt, tgt)
        plt.show()

        print("Decoder Src Layer", layer+1)
        fig, axs = plt.subplots(1, 4, figsize=(20, 10))
        for h in range(4):
            draw(dec_enc_attns[layer][0, h, :len(
                tgt), :len(src)], axs[h], src, tgt)
        plt.show()

In [16]:
# 번역 생성 함수

def evaluate(sentence, model, src_tokenizer, tgt_tokenizer):
    sentence = preprocess_sentence(sentence)

    pieces = src_tokenizer.encode_as_pieces(sentence)
    tokens = src_tokenizer.encode_as_ids(sentence)

    _input = tf.keras.preprocessing.sequence.pad_sequences([tokens],
                                                           maxlen=enc_train.shape[-1],
                                                           padding='post')

    ids = []
    output = tf.expand_dims([tgt_tokenizer.bos_id()], 0)
    for i in range(dec_train.shape[-1]):
        enc_padding_mask, combined_mask, dec_padding_mask = \
            generate_masks(_input, output)

        predictions, enc_attns, dec_attns, dec_enc_attns =\
            model(_input,
                  output,
                  enc_padding_mask,
                  combined_mask,
                  dec_padding_mask)

        predicted_id = \
            tf.argmax(tf.math.softmax(predictions, axis=-1)
                      [0, -1]).numpy().item()

        if tgt_tokenizer.eos_id() == predicted_id:
            result = tgt_tokenizer.decode_ids(ids)
            return pieces, result, enc_attns, dec_attns, dec_enc_attns

        ids.append(predicted_id)
        output = tf.concat(
            [output, tf.expand_dims([predicted_id], 0)], axis=-1)

    result = tgt_tokenizer.decode_ids(ids)

    return pieces, result, enc_attns, dec_attns, dec_enc_attns

In [17]:
# 번역 생성 및 Attention 시각화 결합

def translate(sentence, model, src_tokenizer, tgt_tokenizer, plot_attention=False):
    pieces, result, enc_attns, dec_attns, dec_enc_attns = \
        evaluate(sentence, model, src_tokenizer, tgt_tokenizer)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

    if plot_attention:
        visualize_attention(pieces, result.split(),
                            enc_attns, dec_attns, dec_enc_attns)

In [18]:
# 학습
import random
from tqdm import tqdm_notebook

BATCH_SIZE = 64
EPOCHS = 20

examples = [
    "오바마는 대통령이다.",
    "시민들은 도시 속에 산다.",
    "커피는 필요 없다.",
    "일곱 명의 사망자가 발생했다."
]

for epoch in range(EPOCHS):
    total_loss = 0

    idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm_notebook(idx_list)

    for (batch, idx) in enumerate(t):
        batch_loss, enc_attns, dec_attns, dec_enc_attns = \
            train_step(enc_train[idx:idx+BATCH_SIZE],
                       dec_train[idx:idx+BATCH_SIZE],
                       transformer,
                       optimizer)

        total_loss += batch_loss

        t.set_description_str('Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))

    for example in examples:
        translate(example, transformer, ko_tokenizer, en_tokenizer)

/tmp/ipykernel_67575/2869332712.py:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  t = tqdm_notebook(idx_list)


  0%|          | 0/1011 [00:00<?, ?it/s]

2025-01-20 20:18:31.681052: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f0cc3a27ea0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-01-20 20:18:31.681065: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2025-01-20 20:18:31.683293: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-01-20 20:18:31.689883: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2025-01-20 20:18:31.732249: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Input: 오바마는 대통령이다.
Predicted translation: i , , .
Input: 시민들은 도시 속에 산다.
Predicted translation: i , , .
Input: 커피는 필요 없다.
Predicted translation: i , , .
Input: 일곱 명의 사망자가 발생했다.
Predicted translation: i , , .


  0%|          | 0/1011 [00:00<?, ?it/s]

...

Input: 오바마는 대통령이다.
Predicted translation: joan mccain .
Input: 시민들은 도시 속에 산다.
Predicted translation: people threaten their property line .
Input: 커피는 필요 없다.
Predicted translation: can't pay about you much more caffeine a year .
Input: 일곱 명의 사망자가 발생했다.
Predicted translation: a third person was killed in a town about , people died .


  0%|          | 0/1011 [00:00<?, ?it/s]

Input: 오바마는 대통령이다.
Predicted translation: i finish him .
Input: 시민들은 도시 속에 산다.
Predicted translation: people targeted cities in cities .
Input: 커피는 필요 없다.
Predicted translation: i've been a joke .
Input: 일곱 명의 사망자가 발생했다.
Predicted translation: six people died monday .


  0%|          | 0/1011 [00:00<?, ?it/s]

Input: 오바마는 대통령이다.
Predicted translation: lee will take place .
Input: 시민들은 도시 속에 산다.
Predicted translation: sometimes drifted out of town .
Input: 커피는 필요 없다.
Predicted translation: cnn you need you much more .
Input: 일곱 명의 사망자가 발생했다.
Predicted translation: a half dozen people were killed .
